In [1]:
import pandas

In [2]:
import random

In [3]:
from sklearn.neighbors import KNeighborsClassifier

In [4]:
abalones = pandas.read_excel("dataset_processado.xlsx");
test = pandas.read_excel("app_processado.xlsx");
numNeighbors = 5;

In [5]:
knn = KNeighborsClassifier(n_neighbors = numNeighbors);

In [6]:
#Selecionando índices para teste e treino
trainingIndexes = [];
testIndexes = [];
n = 0;
for item in abalones["sex"]:
    if random.random() > 0.2:
        trainingIndexes.append(n)
    else:
        testIndexes.append(n)
    n+=1

In [7]:
#Criando os dataframes de treino e teste e separando os classificadores
testData = abalones.drop(trainingIndexes);
testClasses = testData["type"];
testData = testData.drop(columns=["type"]);

trainingData = abalones.drop(testIndexes);
classifications = trainingData["type"]
trainingData = trainingData.drop(columns=["type"])

In [8]:
trainingData.count()

sex               2503
length            2503
diameter          2503
height            2503
whole_weight      2503
shucked_weight    2503
viscera_weight    2503
shell_weight      2503
dtype: int64

In [9]:
classifications.count()

2503

In [10]:
testData.count()

sex               629
length            629
diameter          629
height            629
whole_weight      629
shucked_weight    629
viscera_weight    629
shell_weight      629
dtype: int64

In [11]:
#Aplicando K-NN
knn.fit(trainingData, classifications)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [12]:
results = knn.predict(testData);

In [13]:
#Testando acurácia

n = 0;
correctPredictions = 0;
for item in testClasses:
    if item == results[n]:
        correctPredictions += 1;
    n+=1  

In [14]:
print("Acurácia: " + str(correctPredictions/len(results)) + "%")

Acurácia: 0.6375198728139905%
